In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd

In [3]:
train_data = pd.read_csv("/kaggle/input/ml-olympiad-toxic-language-ptbr-detection/train (2).csv")

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16800 entries, 0 to 16799
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    16800 non-null  object
 1   label   16800 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 262.6+ KB


In [5]:
train_data.head()

,text,label
0,"rt @user olha quem chegouuuuu, nossos queridin...",0
1,veio umas teorias muito loucas na minha cabeça...,1
2,@user @user 😂😂😂😂mais nao tinha falado ontem qu...,0
3,rt @user quer ser filha da puta logo comigo qu...,1
4,vai besta 😂😂😂😂 casquei com a ultima foto,1


In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

In [7]:
def clean_text(text):
    # Remove non-alphanumeric characters including emojis
    text = re.sub('[^A-Za-zÀ-ÖØ-öø-ÿ]', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    tokens = word_tokenize(text, language='portuguese')
    # Remove stopwords
    stop_words = set(stopwords.words('portuguese'))
    tokens = [word for word in tokens if word not in stop_words]
    # Stemming
    stemmer = SnowballStemmer('portuguese')
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

train_data['cleaned_text'] = train_data['text'].apply(clean_text)

In [8]:
train_data.head()

,text,label,cleaned_text
0,"rt @user olha quem chegouuuuu, nossos queridin...",0,rt user olha chegouuuuu queridinh vem direçã f...
1,veio umas teorias muito loucas na minha cabeça...,1,vei umas teor louc cabec agor pqp to assust
2,@user @user 😂😂😂😂mais nao tinha falado ontem qu...,0,user user nao fal ontem ia patrocin nad pud vi...
3,rt @user quer ser filha da puta logo comigo qu...,1,rt user quer filh put log comig x pior kkkkkkk...
4,vai besta 😂😂😂😂 casquei com a ultima foto,1,vai best casqu ultim fot


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_data['cleaned_text'], train_data['label'], test_size=0.2, random_state=42)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=500)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

In [12]:
def train_and_evaluate(X_train, X_test, y_train, y_test):
    algorithms = {
        "Logistic Regression": LogisticRegression(),
        "Random Forest": RandomForestClassifier(),
        "Support Vector Machine": SVC(),
        "XGBoost": XGBClassifier()
    }

    for name, model in algorithms.items():
        print("Training", name)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        report = classification_report(y_test, predictions)
        print("Accuracy:", accuracy)
        print("Classification Report:")
        print(report)
        print("-" * 50)

train_and_evaluate(X_train_tfidf, X_test_tfidf, y_train, y_test)

Training Logistic Regression
Accuracy: 0.7386904761904762
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.84      0.78      1881
           1       0.75      0.61      0.67      1479

    accuracy                           0.74      3360
   macro avg       0.74      0.73      0.73      3360
weighted avg       0.74      0.74      0.73      3360

--------------------------------------------------
Training Random Forest
Accuracy: 0.7366071428571429
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.79      0.77      1881
           1       0.71      0.67      0.69      1479

    accuracy                           0.74      3360
   macro avg       0.73      0.73      0.73      3360
weighted avg       0.74      0.74      0.74      3360

--------------------------------------------------
Training Support Vector Machine
Accuracy: 0.74375
Classification Report:
              p

In [13]:
test_data = pd.read_csv("/kaggle/input/ml-olympiad-toxic-language-ptbr-detection/test (4).csv")

In [14]:
test_data.head()

,id,text
0,0,@user nossa mano te odeio na moral kkkkkkkkkkk...
1,1,@user @user a edição ficou muito boa! kkkkkkkk...
2,2,"@user largada vá, visto que a vaca anda à solta 😂"
3,3,"poxa, eu queria ganhar um boné. alguém me da u..."
4,4,"@user amiga, tudo bem, não precisa se desculpa..."


In [15]:
def clean_text(text):
   
    text = re.sub('[^A-Za-zÀ-ÖØ-öø-ÿ]', ' ', text)
    text = text.lower()
    tokens = word_tokenize(text, language='portuguese')
    stop_words = set(stopwords.words('portuguese'))
    tokens = [word for word in tokens if word not in stop_words]
    stemmer = SnowballStemmer('portuguese')
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

test_data['cleaned_text'] = test_data['text'].apply(clean_text)

In [16]:
test_data.head()

,id,text,cleaned_text
0,0,@user nossa mano te odeio na moral kkkkkkkkkkk...,user man odei moral kkkkkkkkkkkkkkkkkkk
1,1,@user @user a edição ficou muito boa! kkkkkkkk...,user user ediçã fic boa kkkkkkkkkkk falt luk tim
2,2,"@user largada vá, visto que a vaca anda à solta 😂",user larg vá vist vac anda solt
3,3,"poxa, eu queria ganhar um boné. alguém me da u...",pox quer ganh bon alguém bon
4,4,"@user amiga, tudo bem, não precisa se desculpa...",user amig tud bem precis desculp escrot q ente...


In [17]:
test = tfidf_vectorizer.transform(test_data['cleaned_text'])

In [18]:
model = XGBClassifier()
model.fit(X_train_tfidf, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [19]:
test_predictions = model.predict(test)

In [20]:
# Assuming 'id' column contains the unique identifiers for each sample in the test data
test_ids = test_data['id']

# Save predictions to a CSV file
submission_df = pd.DataFrame({'id': test_ids, 'label': test_predictions})
submission_df.to_csv('submission.csv', index=False)